In [1]:
%pip install -qr https://raw.githubusercontent.com/anton-dergunov/hugging-face-deep-rl/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch
import cv2
from IPython.display import Video
import imageio
import os

### Create Environment

In [3]:
# Create a vectorized environment (parallel environments)
# "CartPole-v1" is a simple classic control task
# n_envs=4 means we run 4 copies of the environment in parallel
vec_env = make_vec_env("CartPole-v1", n_envs=4)

### Define & Train PPO Agent

In [4]:
# Initialize PPO agent with Multi-Layer Perceptron policy
# verbose=1 shows training logs
model = PPO("MlpPolicy", vec_env, verbose=1)

# Train the agent for 25,000 timesteps
model.learn(total_timesteps=25000)

# Save the trained model
MODEL_PATH = "models/ppo_cartpole"
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
model.save(MODEL_PATH)

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.4     |
|    ep_rew_mean     | 22.4     |
| time/              |          |
|    fps             | 20669    |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 33.2         |
|    ep_rew_mean          | 33.2         |
| time/                   |              |
|    fps                  | 7639         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0140280835 |
|    clip_fraction        | 0.2          |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.682       |
|    explained_variance   | -0.00234     

### Load Trained Model

In [5]:
# Delete current model from memory
del model

# Load the saved model
model = PPO.load(MODEL_PATH)

### Run Trained Agent

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

frames = []
obs, info = env.reset()
episode, step = 0, 0

for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    frame = env.render()

    cv2.putText(frame, f"Episode: {episode}", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f"Step: {step}", (10, 55),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    frames.append(frame)
    step += 1

    if terminated or truncated:
        episode += 1
        step = 0
        obs, info = env.reset()

env.close()

# Save to video
VIDEO_PATH = "videos/cartpole_demo.mp4"
os.makedirs(os.path.dirname(VIDEO_PATH), exist_ok=True)
imageio.mimsave(VIDEO_PATH, frames, fps=30, macro_block_size=None)

Render mode: rgb_array


TypeError: Wrapper.render() got an unexpected keyword argument 'mode'

In [ ]:
Video(VIDEO_PATH, embed=True)